# Cats vs Dogs 

In [20]:
import cv2
import pandas as pd
import numpy as np
import os               
from random import shuffle
from tqdm import tqdm 
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import os
import random
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import  KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.metrics import recall_score, confusion_matrix, precision_score, f1_score, accuracy_score, classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_classification
from sklearn.ensemble import VotingClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [21]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


**Lets resize all of our pictures and turn them into A dataset.**

In [33]:
TRAIN_DIRECTORY= r'C:\Users\yasmi\OneDrive\Desktop\dogs-vs-cats\train' 
IMG_SIZE = 50
LR = 1e-3

MODEL_NAME = 'dogsvscats-{}-{}.model'.format(LR, '2conv-basic')

In [34]:
def label_img(img):
    word_label = img.split('.')[-3]
   
    if word_label == 'cat': return 1
    #                             
    elif word_label == 'dog': return 0

In [35]:
labal_data =[]
training_data = []
for img in tqdm(os.listdir(TRAIN_DIRECTORY)):
        label = label_img(img)
        path = os.path.join(TRAIN_DIRECTORY,img)
        img = cv2.imread(path, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        training_data.append(pd.Series(img.flatten()))
        labal_data.append(label)

100%|██████████| 6708/6708 [00:19<00:00, 348.05it/s]


In [36]:
images_df = pd.DataFrame(training_data)
labels_df = pd.DataFrame(labal_data, columns = ['labels'])

In [37]:
df = pd.concat([images_df, labels_df], axis=1)

# Logistic Regression Model

In [38]:
X = df.drop(['labels'],axis=1)
Y = df['labels']

In [42]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.3, random_state=120)
print("Xtrain : ",Xtrain.shape)
print("Xtest : ",Xtest.shape)
print("Ytrain : ",Ytrain.shape)
print("Ytest : ",Ytest.shape)

Xtrain :  (4695, 7500)
Xtest :  (2013, 7500)
Ytrain :  (4695,)
Ytest :  (2013,)
